# 1) IMPORT

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import sparse
import os

from sklearn.compose import make_column_transformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

# 2) DATA PROCESSING

# Kind of data spliting (already split)
from problem import get_train_data, get_test_data
X_train, y_train = get_train_data()
X_test, y_test = get_test_data()

print(f"Intialement la taille de X_train est {X_train.shape} et celle de y_train {y_train.shape}")
print(f"Intialement la taille de X_test est {X_test.shape} et celle de y_test {y_test.shape}")

# Data reducing
def decrease_data_size(X, y):
    # decrease datasize
    X = X.reset_index(drop=True)  # make sure the indices are ordered
    # 1. take only first 2 subjects
    subjects_used = np.unique(X['subject'])[:2]
    X = X.loc[X['subject'].isin(subjects_used)]

    # 2. take only n_samples from each of the subjects
    n_samples = 100  # use only 100 samples/subject
    X = X.groupby('subject').apply(
        lambda s: s.sample(n_samples, random_state=42))
    X = X.reset_index(level=0, drop=True)  # drop subject index

    # get y corresponding to chosen X
    y = y[X.index]
    
    return X, y

X_train, y_train = decrease_data_size(X_train, y_train)
X_test, y_test = decrease_data_size(X_test, y_test)

print(f"Après réduction, la taille de X_train est {X_train.shape} et celle de y_train {y_train.shape}")
print(f"Après réduction, la taille de X_test est {X_test.shape} et celle de y_test {y_test.shape}")


# 3) MODEL KNN

In [3]:
N_JOBS = -1

def get_estimator():

    # K-nearest neighbors
    clf = KNeighborsClassifier(n_neighbors=3,algorithm='brute')
    kneighbors = MultiOutputClassifier(clf, n_jobs=N_JOBS)

    preprocessor = make_column_transformer(('drop', ['subject', 'L_path']), #drop columns with type objects
                                           remainder='passthrough')

    pipeline = Pipeline([
        ('transformer', preprocessor),
        ('classifier', kneighbors)
    ])

    return pipeline

In [4]:
pipeline = get_estimator()
pipeline.fit(X_train, y_train)
y_pred_knn = pipeline.predict(X_test)

## PERFORMANCES

### 1) Jaccard error

In [5]:
# Jaccard high = predicteur nul
from sklearn.metrics import jaccard_score

score_knn = 1 - jaccard_score(y_test, y_pred_knn, average='samples')
print(f"The jaccard error for KNN is {score_knn}")

The jaccard error for KNN is 0.995


## 2) 

# MODEL LASSO

In [6]:
path_subject_1 = X_train[X_train['subject'] == 'subject_1']['L_path'].iloc[0]
path_subject_2 = X_train[X_train['subject'] == 'subject_2']['L_path'].iloc[0]

lead_field_1 = np.load(path_subject_1)
lead_field_2 = np.load(path_subject_2)
print(lead_field_1.files)

#lead_field['lead_field'] is of shape (n_sensors, n_locations).
#lead_field['parcel_indices'] is of shape n_locations and tells us in wchich part of brain is the localisation
print(lead_field_1["lead_field"].shape)
print(lead_field_1["parcel_indices"].shape)

print(lead_field_2["lead_field"].shape)
print(lead_field_2["parcel_indices"].shape)

['lead_field', 'parcel_indices', 'signal_type']
(204, 4688)
(4688,)
(204, 4690)
(4690,)


In [7]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.base import TransformerMixin


def _get_coef(est):
    """Get coefficients from a fitted regression estimator."""
    if hasattr(est, 'steps'):
        return est.steps[-1][1].coef_
    return est.coef_


class SparseRegressor(BaseEstimator, ClassifierMixin, TransformerMixin):
    '''Provided regression estimator (ie model) solves inverse problem
        using data X and lead field L. The estimated coefficients (est_coef
        sometimes called z) are then used to predict which parcels are active.

        X must be of a specific structure with a column name 'subject' and
        'L_path' which gives the path to lead_field files for each subject
    '''
    def __init__(self, model, n_jobs=1):
        self.model = model
        self.n_jobs = n_jobs
        self.parcel_indices = {}
        self.Ls = {}

    def fit(self, X, y):
        return self

    def predict(self, X):
        return (self.decision_function(X) > 0).astype(int)

    def _run_model(self, model, L, X):
        norms = np.linalg.norm(L, axis=0)
        L = L / norms[None, :]

        est_coefs = np.empty((X.shape[0], L.shape[1]))
        for idx, idx_used in enumerate(X.index.values):
            x = X.iloc[idx].values
            model.fit(L, x)
            est_coef = np.abs(_get_coef(model))
            est_coef /= norms
            est_coefs[idx] = est_coef

        return est_coefs.T

    def decision_function(self, X):
        X = X.reset_index(drop=True)

        for subject_id in np.unique(X['subject']):
            if subject_id not in self.Ls:
                # load corresponding L if it's not already in
                L_used = X[X['subject'] == subject_id]['L_path'].iloc[0]
                lead_field = np.load(L_used)
                self.parcel_indices[subject_id] = lead_field['parcel_indices']

                # scale L to avoid tiny numbers
                self.Ls[subject_id] = 1e8 * lead_field['lead_field']
                assert (self.parcel_indices[subject_id].shape[0] ==
                        self.Ls[subject_id].shape[1])

        n_parcels = np.max([np.max(s) for s in self.parcel_indices.values()])
        betas = np.empty((len(X), n_parcels))
        for subj_idx in np.unique(X['subject']):
            L_used = self.Ls[subj_idx]

            X_used = X[X['subject'] == subj_idx]
            X_used = X_used.drop(['subject', 'L_path'], axis=1)

            est_coef = self._run_model(self.model, L_used, X_used)

            beta = pd.DataFrame(
                np.abs(est_coef)
            ).groupby(self.parcel_indices[subj_idx]).max().transpose()
            betas[X['subject'] == subj_idx] = np.array(beta)
        return betas

In [8]:
# we change alpha for better result
from sklearn.base import RegressorMixin

class CustomSparseEstimator(BaseEstimator, RegressorMixin):
    """ Regression estimator which uses LassoLars algorithm with given alpha
        normalized for each lead field L and x. """
    def __init__(self, alpha=0.2):
        self.alpha = alpha

    def fit(self, L, x):
        alpha_max = abs(L.T.dot(x)).max() / len(L)
        alpha = self.alpha * alpha_max
        lasso = linear_model.LassoLars(alpha=alpha, max_iter=3,
                                       normalize=False, fit_intercept=False)
        lasso.fit(L, x)
        self.coef_ = lasso.coef_

In [9]:
from sklearn import linear_model

model_lars = linear_model.LassoLars(alpha=1.0, max_iter=3,
                                    normalize=False,
                                    fit_intercept=False)

lasso_lars = SparseRegressor(model_lars)
lasso_lars.fit(X_train, y_train)
y_pred_lassolars = lasso_lars.predict(X_test)

In [10]:
custom_model = CustomSparseEstimator(alpha=0.2)
lasso_lars_alpha = SparseRegressor(custom_model)
lasso_lars_alpha.fit(X_train, y_train)
y_pred_alpha = lasso_lars_alpha.predict(X_test)

## PERFORMANCE
### 1) JACCARD

In [11]:
score_lassolars = 1 - jaccard_score(y_test, y_pred_lassolars, average='samples')
print(f'Jaccard error for the first Lasso Lars algorithm using lead fields is {score_lassolars}')

Jaccard error for the first Lasso Lars algorithm using lead fields is 1.0


In [12]:
score_lassolars = 1 - jaccard_score(y_test, y_pred_alpha, average='samples')
print(f'Jaccard error for the custom Lasso Lars algorithm using lead fields is {score_lassolars}')

Jaccard error for the custom Lasso Lars algorithm using lead fields is 0.7251666666666667


## CHOIX DU ALPHA

In [20]:
L,Predictions,Score=[],[],[]
for i in np.arange(0,1,0.01):
    elt=CustomSparseEstimator(alpha=i)
    L.append(elt)
for alph_model in L :
    asso_alpha = SparseRegressor(alph_model)
    lasso_alpha.fit(X_train, y_train)
    Predictions.append(lasso_alpha.predict(X_test))
for pred in Predictions:
    Score.append(1 - jaccard_score(y_test, pred, average='samples'))

In [21]:
print("le alpha optimale est")
print(np.argmin(Score)*0.01)

le alpha optimale est
0.0
